In [1]:
#DC OPF model abstraction

#Import libraries
import numpy as np
import pandas as pd 

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

#load data
xls = pd.ExcelFile('data\case39.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')


#Y matrix
base = 100
bus_len = bus.shape[0]
Ybus = np.zeros([bus_len,bus_len])
gen_len = len(gen['bus'].ravel())
x = [pd.DataFrame() for i in range(bus_len)]

for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/pd.DataFrame(x[i]['x']).iloc[j][0]
        Ybus[fro.loc[j][0]-1,i-1] =  -1/pd.DataFrame(x[i]['x']).iloc[j][0]

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1)


# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2

#Create a model
model = pyo.ConcreteModel()

model.IDX1 = range(gen_len)
model.IDX2 = range(bus_len)
model.Pg = pyo.Var(model.IDX2)
model.Qg = pyo.Var(model.IDX2)
model.t = pyo.Var(model.IDX2)

for i in model.IDX2:
    model.Pg[i].fix(0) 
    model.Qg[i].fix(0) 
    
for i in model.IDX1:
    model.Pg[gen['bus'][i]-1].fixed = False
    model.Qg[gen['bus'][i]-1].fixed = False



# declare constraints
model.c = pyo.ConstraintList()
model.c.add(expr = sum(model.Pg[i] for i in model.IDX2) == sum(Pd))
for i in range(bus_len):
    model.c.add(expr = model.Pg[i] <= Pmax[i])
    model.c.add(expr = model.Pg[i] >= Pmin[i])

#Nodal equations
for i in range(bus_len):
    model.c.add(expr = sum([Ybus[i,j]*model.t[j] for j in range(bus_len)]) - model.Pg[i] + Pd[i]== 0)
    
#Line flow constraints
for i in range(len(branch)):
    x = int(branch.loc[i]['fbus'])-1
    y = int(branch.loc[i]['tbus'])-1
    model.c.add(expr = (Ybus[x,y])* (model.t[x]-model.t[y]) <= int(branch.loc[i]['rateA'])/base)
    model.c.add(expr = (Ybus[x,y])* (model.t[y]-model.t[x]) <= int(branch.loc[i]['rateA'])/base)


#model.c.add(expr = model.t[0] == 0)    
# declare objective
model.cost = pyo.Objective( 
    expr = sum(cost['cons']) + 
    (sum(model.Pg[i]*cost_lin[i] for i in model.IDX2)) + 
    (sum(cost_quad[i]*model.Pg[i]**2 for i in model.IDX2))
)    
# solve
SolverFactory('gurobi').solve(model).write()


#Print results
print('Cost = ',model.cost())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: 
  Lower bound: 41263.940785800776
  Upper bound: 41263.940785800776
  Number of objectives: 1
  Number of constraints: 211
  Number of variables: 50
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 50
  Number of nonzeros: 356
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was so